In [2]:
# CREATED: 4-APR-2022
# LAST EDIT: 10-MAY-2022
# AUTHOR(S): DUANE RINEHART
'''
IMPORT POLYGON POINTS FROM CSV (EVENTUALLY PULL FROM RDBMS)
Note: polygons_dump5.csv GENERATED WITH QUERY - SELECT * FROM annotations_points WHERE (prep_id='DK55' AND (label='7N_L' OR label='5N_L')) OR (prep_id='MD594' AND (label='SC' OR label='IC'))
'''
import os
import pandas as pd
import numpy as np

win_data_dir = 'E:/dev/'
linux_data_dir = '/mnt/e/dev/'


'''
SET SRC FOLDER BASED (CROSS-PLATFORM)
'''
if os.name == "nt":
    input_fullPath = os.path.join(win_data_dir)
else:
    input_fullPath = os.path.join(linux_data_dir)

infile = os.path.join(input_fullPath, "MD585_dump5.csv")
outfile = os.path.join(input_fullPath, "output_for_Yoav_testdel.xlsx")

def load_raw_data(infile):
    return pd.read_csv(infile, sep='\t', index_col=0)


def parse_df(df_raw_data, prep_id, label, xy_resolution_microns_to_pixels, z_resolution_section_to_microns):
    data = df_raw_data[np.logical_and(df_raw_data.prep_id==prep_id,df_raw_data.label==label)]
    data['x'] = data['x'].apply(lambda x: x / xy_resolution_microns_to_pixels)
    data['y'] = data['y'].apply(lambda y: y / xy_resolution_microns_to_pixels)
    data['z'] = data['z'].apply(lambda z: z / z_resolution_section_to_microns)
    data["unique_key"] =  data["prep_id"].astype(str) + '_' + data["label"].astype(str) + '_' + data["z"].round().astype(int).astype(str)
    data["coord"] = list(zip(data['x'], data["y"], data["z"]))
    data[(data['prep_id'] == prep_id) & (data['label'] == label)]
    data = data.rename(columns={'ordering': 'polygon_ordering'})

    #RECODE, IF NECESSARY
    data.loc[data["FK_structure_id"] == 54, "FK_structure_id"] = "POLYGON"
    data.loc[data["FK_owner_id"] == 4, "FK_owner_id"] = "dk"
    data.loc[data["FK_owner_id"] == 37, "FK_owner_id"] = "beth"
    data.loc[data["FK_input_id"] == 1, "FK_input_id"] = "manual person"


    selected_columns = data[
        ["unique_key", "prep_id", "label", "coord", "z", "polygon_id", "volume_id", "polygon_ordering"]
    ]  # EXTRACT SPECIFIC COLUMNS FOR ANALYSIS

    # selected_columns = data[
    #     ["unique_key", "prep_id", "label", "coord", "z", "polygon_ordering", 'volume_ordering']
    # ]  # EXTRACT SPECIFIC COLUMNS FOR ANALYSIS

    return selected_columns


df_raw_data = load_raw_data(infile) # REPLACE W/ RDBMS CALL FOR PRODUCTION


# FILTERING CONSTANTS
z_resolution_section_to_microns = 20 # STORED IN DB

# POLYGON SETS BY prep_id, label CREATED INDIVIDUALLY BELOW FOR TESTING
prep_id = 'MD594'
xy_resolution_microns_to_pixels = .452 # STORED IN DB (FOR FOUNDATION BRAINS)
label = 'SC'
extracted_columns_MD594_SC = parse_df(df_raw_data, prep_id, label, xy_resolution_microns_to_pixels, z_resolution_section_to_microns)
label = 'IC'
extracted_columns_MD594_IC = parse_df(df_raw_data, prep_id, label, xy_resolution_microns_to_pixels, z_resolution_section_to_microns)
prep_id = 'DK55'
xy_resolution_pixels_microns = .325 # STORED IN DB (FOR DK55)
label = '7N_L'
extracted_columns_DK55_7N_L = parse_df(df_raw_data, prep_id, label, xy_resolution_microns_to_pixels, z_resolution_section_to_microns)
label = '7N_R'
extracted_columns_DK55_7N_R = parse_df(df_raw_data, prep_id, label, xy_resolution_microns_to_pixels, z_resolution_section_to_microns)

# EACH DATAFRAME CONTAINS LIST OF TUPLES (coord) THAT MAY BE USED FOR PLOTTING POLYGONS IN NEUROGLANCER (SOME EXAMPLES IN NOTEBOOK - BELOW)

# FOLLOWING CODE ONLY USEFUL IF BATCH PROCESSING ALL DATA (HOWEVER FOR INTEGRATION IN APP (PIPELINE), LIKELY DB QUERY WITH LOOP IS SUFFICIENT)

consolidated_coordinates = pd.concat([extracted_columns_MD594_SC, extracted_columns_MD594_IC, extracted_columns_DK55_7N_L, extracted_columns_DK55_7N_R])

#ENCODE VOLUME ID AS CATEGORICAL VARIABLE
consolidated_coordinates['volume_ordering'] = pd.factorize(consolidated_coordinates['volume_id'])[0]

# SAVE DATA FOR YOAV INPUTS
consolidated_coordinates.to_excel(outfile, sheet_name='polygons_volumes')

# # CREATE DICTIONARY OF COORDINATES
# transformed_polygon_structures = consolidated_coordinates.groupby(['unique_key'])['coord'].apply(lambda x: list(np.unique(x))).to_dict()
#
# # OUTPUT FORMAT:
# # -KEY IS CONCATENATED prep_id_structure_section ("_" delimiter)
# # -VALUE IS LIST OF COORDINATE TUPLES (x, y, z)
# #
# # TO ACCESS [EXAMPLE WITH prep_id 'MD594', label 'IC', section 140]:
# polygon_index = 'MD594_IC_140'
# for coordinate_list in transformed_polygon_structures[polygon_index]:
#     print(coordinate_list)

/tmp/ipykernel_21614/678556094.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['x'] = data['x'].apply(lambda x: x / xy_resolution_microns_to_pixels)
/tmp/ipykernel_21614/678556094.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['y'] = data['y'].apply(lambda y: y / xy_resolution_microns_to_pixels)
/tmp/ipykernel_21614/678556094.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

ModuleNotFoundError: No module named 'openpyxl'

In [62]:

consolidated_coordinates['volume_ordering'] = pd.factorize(consolidated_coordinates['polygon_id'])[0]
consolidated_coordinates['volume_ordering'].unique()
#data = df_raw_data[np.logical_and(df_raw_data.prep_id==prep_id,df_raw_data.label==label)]

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [217]:
# TO VIEW RANGE OF SECTION VALUES FOR BRAIN/STRUCTURE (STORED SEPARATELY)
extracted_columns_MD594_SC.z.unique()

array([161., 157., 248., 173., 164., 279., 209., 268., 274., 200., 263.,
       176., 146., 288., 296., 189., 196., 165., 212., 186., 160., 214.,
       217., 178., 287., 292., 202., 139., 218., 199., 163., 219., 242.,
       260., 142., 272., 188., 179., 175., 144., 152., 137., 205., 167.,
       251., 203., 235., 305., 285., 184., 195., 228., 266., 236., 239.,
       270., 282., 267., 241., 148., 306., 162., 197., 244., 291., 169.,
       293., 194., 280., 229., 230., 149., 143., 256., 177., 147., 180.,
       245., 150., 172., 231., 204., 304., 246., 275., 171., 249., 181.,
       207., 286., 201., 257., 281., 237., 259., 182., 262., 278., 250.,
       254., 158., 215., 232., 252., 225., 301., 307., 276., 233., 154.,
       289., 166., 309., 210., 269., 192., 255., 265., 308., 299., 220.,
       298., 243., 226., 138., 170., 159., 271., 213., 211., 284., 300.,
       221., 290., 198., 141., 185., 206., 190., 223., 151., 247., 153.,
       155., 297., 253., 208., 140., 227., 183., 22

In [215]:
# TO VIEW VOLUME IDS FOR SPECIFIC STRUCTURE (EACH PREP_ID AND BRAIN STRUCTURE STORED SEPARATELY)
extracted_columns_MD594_SC.volume_id

id
242321    9a661f4f149f6571e0873cf2182d12f5a54a2452
242322    9a661f4f149f6571e0873cf2182d12f5a54a2452
242323    9a661f4f149f6571e0873cf2182d12f5a54a2452
242324    9a661f4f149f6571e0873cf2182d12f5a54a2452
242325    9a661f4f149f6571e0873cf2182d12f5a54a2452
                            ...                   
264258    9a661f4f149f6571e0873cf2182d12f5a54a2452
264259    9a661f4f149f6571e0873cf2182d12f5a54a2452
264260    9a661f4f149f6571e0873cf2182d12f5a54a2452
264261    9a661f4f149f6571e0873cf2182d12f5a54a2452
264262    9a661f4f149f6571e0873cf2182d12f5a54a2452
Name: volume_id, Length: 21942, dtype: object

In [3]:
consolidated_coordinates['volume_ordering'].unique()

array([0, 1, 2, 3])